## Data Preparation

In [36]:
from __future__ import print_function, division
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from astropy.io import fits

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
%pylab inline
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import foolbox
import torch
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from scipy.ndimage import gaussian_filter
from os.path import join as oj
import sys
sys.path.append('../visualization')
sys.path.append('../acd/util')
sys.path.append('../acd/scores')
sys.path.append('../acd/agglomeration')
from tqdm import tqdm
import viz_2d as viz
import cd
import score_funcs
import bandpass_filter
# plt.style.use('dark_background')

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from data import *
data_path = '/scratch/users/vision/data/cosmo'
mnu_dataset = MassMapsDataset(oj(data_path, 'cosmological_parameters.txt'),  oj(data_path, 'z1_256'))

In [ ]:
plt.figure(figsize=(15,5))
for i in range(len(mnu_dataset)):
    sample = mnu_dataset[i]

    #print(i, sample['image'].shape, sample['params'].shape)

    subplot(1, 4, i + 1)
    tight_layout()
    axis('off')
    imshow(squeeze(sample['image']), cmap='magma',vmax=0.15,vmin=-0.05 )
    title(r"$m_\nu$=%0.2f; $\Omega_m$=%0.2f; $10^9A_s$=%0.2f"%(sample['params'][0], sample['params'][1], sample['params'][2] ) )

    if i == 3:
        break

## Load the model

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
import time
import os
import copy

model_ft = models.resnet18(pretrained=False)

# Modifying the model to predict the three cosmological parameters from single channel images
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 3)
model_ft = model_ft.to(device)

# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=10)

# model_ft = model_ft.load_state_dict(oj(data_path, 'resnet18_state_dict'))
model_ft.load_state_dict(torch.load(oj(data_path, 'resnet18_state_dict')))

cuda:0


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [60]:
data_transform = transforms.Compose([
        ToTensor()
    ])
mnu_dataset = MassMapsDataset(oj(data_path, 'cosmological_parameters.txt'),  
                              oj(data_path, 'z1_256'),
                              transform=data_transform)
# mnu_dataset = MassMapsDataset('cosmological_parameters.txt',
#                               'z1_256', 
#                               transform=data_transform)

dataloader = torch.utils.data.DataLoader(mnu_dataset, batch_size=64, 
                                         shuffle=True, num_workers=4)

# check that it makes reasonable preds

In [41]:
with torch.no_grad():
    for data in dataloader:
        inputs, params = data['image'], data['params']
        inputs = inputs.to(device)
        params = params.to(device)
        outputs = model_ft(inputs)
        break

In [ ]:
scatter(params.cpu()[:,1], outputs.cpu()[:,1])
xlabel('true param')
ylabel('predicted param')

# analyze one prediction

In [ ]:
sample = mnu_dataset[1]
plt.figure(dpi=200)
sample = mnu_dataset[i]

#print(i, sample['image'].shape, sample['params'].shape)

# subplot(1, 4, i + 1)
# tight_layout()
axis('off')
im = np.squeeze(sample['image'])
plt.imshow(im, cmap='magma', vmax=0.15, vmin=-0.05)
plt.show()

# imshow(squeeze(sample['image']), cmap='magma',vmax=0.15,vmin=-0.05 )
# title(r"$m_\nu$=%0.2f; $\Omega_m$=%0.2f; $10^9A_s$=%0.2f"%(sample['params'][0], sample['params'][1], sample['params'][2] ) )

# if i == 3:
#     break

In [55]:
import util

In [56]:
t = util.Transforms()

In [ ]:
t.sigma = 2e-1 # [0, 1]
# t.sigma = 1e-1
# t.sigma = 1e-9
# transform = t.gaussian_filter
# transform = t.bandpass_filter_approx
transform = t.bandpass_filter
util.visualize(im, transform)

In [83]:
im_t = torch.Tensor(im.reshape(1, 1, 256, 256)).to(device) #.astype('float32'))
model_ft(im_t).shape

torch.Size([1, 3])

In [ ]:
util.visualize(im, transform)